<a href="https://colab.research.google.com/github/LenSin3/big-data-challenge/blob/main/amz_review_level_1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [44.4 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-02-01 21:30:32--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.09s   

2021-02-01 21:30:32 (9.91 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonPCReviews").getOrCreate()

# **Extract**

In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz'
spark.sparkContext.addFile(url)
pc_review_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_PC_v1_00.tsv.gz"), sep=r'\t', header=True)

# Show DataFrame
pc_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...|              PC|          5|    

In [7]:
# Count number of rows in dataset
num_rows = pc_review_df.count()
print(f'The are {num_rows} records in the dataset')

The are 6908554 records in the dataset


In [8]:
# Create data frame for overall reviews
# Create dataframe with the following columns review_id, customer_id, product_id, product_parent, review_date
review_df = pc_review_df.select('review_id', 'customer_id', 'product_id', 'product_parent', 'review_date')
review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3ARRMDEGED8RD|   22873041|B00KJWQIIC|     335625766| 2015-08-31|
| RQ28TSA020Y6J|   30088427|B013ALA9LA|     671157305| 2015-08-31|
| RUXJRZCT6953M|   20329786|B00PML2GQ8|     982036237| 2015-08-31|
| R7EO0UO6BPB71|   14215710|B001NS0OZ4|     576587596| 2015-08-31|
|R39NJY2YJ1JFSV|   38264512|B00AQMTND2|     964759214| 2015-08-31|
|R31SR7REWNX7CF|   30548466|B00KX4TORI|     170101802| 2015-08-31|
| RVBP8I1R0CTZ8|     589298|B00P17WEMY|     206124740| 2015-08-31|
|R1QF6RS1PDLU18|   49329488|B00TR05L9Y|     778403103| 2015-08-31|
|R23AICGEDAJQL1|   50728290|B0098Y77OG|     177098042| 2015-08-31|
|R2EY3N4K9W19UP|   37802374|B00IFYEYXC|     602496520| 2015-08-31|
| RC9AW4HKJ016M|   52027882|B0091ITP0S|     977217357| 2015-08-31|
|R2ALWJE9N6ZBXD|   41770239|B008I21EA2|     295632907| 2015-08

In [9]:
# Create data frame for products
products_df = pc_review_df.select('product_id', 'product_title')
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00KJWQIIC|Plemo 14-Inch Lap...|
|B013ALA9LA|TP-Link OnHub AC1...|
|B00PML2GQ8|AmazonBasics USB ...|
|B001NS0OZ4|Transcend P8 15-i...|
|B00AQMTND2|Aleratec SATA Dat...|
|B00KX4TORI|Kingston Digital ...|
|B00P17WEMY|White 9 Inch Unlo...|
|B00TR05L9Y|Lenovo TAB2 A10 -...|
|B0098Y77OG|                Acer|
|B00IFYEYXC|AzureWave Broadco...|
|B0091ITP0S|HDE Rotating iPad...|
|B008I21EA2|Linksys AC1750 Wi...|
|B00MRB7SBO|iPad Pro 9.7, iPa...|
|B00LLER2CS|SanDisk 16GB CZ43...|
|B00B0CQCCC|TRENDnet Wireless...|
|B00GU8W5AE|Redragon M901 PER...|
|B00XHMXJQ0|Mudder MHL Adapte...|
|B00YGJJQ6U|Fintie iPad Air 2...|
|B00XMN20Y6|Fintie iPad 2/3/4...|
|B00MA40W9I|Egoway® New Lapto...|
+----------+--------------------+
only showing top 20 rows



In [10]:
from pyspark.sql.functions import min, max, count

In [11]:
# Create data frame for customers
customers_df = pc_review_df.groupBy('customer_id').count()
customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   21374977|    1|
|   16281356|    1|
|   22073787|    1|
|   10245571|    7|
|   37757447|   16|
|   21977184|    2|
|     112911|    1|
|   15259255|    1|
|   27642109|    1|
|   41826621|    1|
|   38289873|   13|
|   10494993|    1|
|   27435626|    6|
|   41534903|    1|
|    1975358|    2|
|   22574219|    1|
|   22279960|    1|
|   47399638|    5|
|   22848554|    1|
|   51068983|   12|
+-----------+-----+
only showing top 20 rows



In [12]:
# Update column name from count to customer_count
customers_df_renamed = customers_df.withColumnRenamed('count','customer_count')
customers_df_renamed.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   21374977|             1|
|   16281356|             1|
|   22073787|             1|
|   10245571|             7|
|   37757447|            16|
|   21977184|             2|
|     112911|             1|
|   15259255|             1|
|   27642109|             1|
|   41826621|             1|
|   38289873|            13|
|   10494993|             1|
|   27435626|             6|
|   41534903|             1|
|    1975358|             2|
|   22574219|             1|
|   22279960|             1|
|   47399638|             5|
|   22848554|             1|
|   51068983|            12|
+-----------+--------------+
only showing top 20 rows



# **Load**

In [13]:
review_lines = pc_review_df.select('review_body')
review_lines.show(5)

+--------------------+
|         review_body|
+--------------------+
|I was very surpri...|
|I am a Google emp...|
|Bought cables in ...|
|nope, cheap and slow|
|Excellent! Great ...|
+--------------------+
only showing top 5 rows



In [ ]:
# load review_df

    
      

In [ ]:
# Create 
